<a href="https://colab.research.google.com/github/YashBurman1/MiniProjects/blob/main/Reading_PDFs_Using_PY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing And Installing Necessary Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 14.8 MB/s eta 0:00:00


In [ ]:
import PyPDF2

# **Importing the pdf containing the Data**



the dataset can be dowloaded from:
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00020.htm


In [ ]:
pdf_file_obj = open("/content/U.S. Senate_ U.S. Senate Roll Call Votes 117th Congress - 1st Session (1).pdf","rb")

# **Extracting the Data from the PDF and Cleaning the Data** 

In [ ]:
pdfReader = PyPDF2.PdfReader(pdf_file_obj)

In [ ]:
len(pdfReader.pages)

5

In [ ]:
pageObj = pdfReader.pages[0]

In [ ]:
pageObj2 = pdfReader.pages[1]

In [ ]:
page_Content = pageObj.extract_text()
page_content = page_Content.split(" ")
page_content

In [ ]:
page_Content2 = pageObj2.extract_text()
page_content2 = page_Content2.split(" ")

In [ ]:
content_list = []
for i in page_content:
    content_list.append(i.strip(","))


In [ ]:
content_list2 = []
for i in page_content2:
    content_list2.append(i.strip(","))


In [ ]:
i = 0
while content_list[i] != 'Name\xa0\nBaldwin':
  i+= 1
start_index = i


In [ ]:
j = 0
for j in range(len(content_list2)):
  if content_list2[j] == "StateMurray":
    start_index2 = j
  if content_list2[j] == "Grouped":
    end_index2 = j-1

In [ ]:
content_list = content_list[(start_index):]
content_list2 = content_list2[start_index2:end_index2]

In [ ]:
content_list[0] = "Baldwin"
content_list2[0] = "Murray"

In [ ]:
content_list

In [ ]:
content_list2

In [ ]:
content = content_list + content_list2 


In [ ]:
with open("content.txt","w") as file:
  file.writelines(content)

# **Forming a dictionary with Senators' Name as Keys and their Party, the state they represent and their Votes resp. as Values**   

In [ ]:
Master_Dict = {}
Senator_Names = []
Political_Affiliation = []
Yea_Nea = []
State = []
with open("content.txt","r") as c_f:
  Senator_Name = ""
  Sen_Pol = ""
  Sen_State = ""
  Sen_Vote = 0
  for i in range(100):
    record = c_f.readline() 
    for j in range(len(record)):
      if record[j] == "(":
        index_ = j
    Senator_Name = record[:index_]
    Sen_Pol = record[index_+1]
    Sen_State = record[index_+3:index_+5]
    if record[index_+6] == "N":
      Sen_Vote = 0
    else:
      Sen_Vote = 1
    Senator_Names.append(Senator_Name)
    Political_Affiliation.append(Sen_Pol)
    Yea_Nea.append(Sen_Vote)
    State.append(Sen_State)
    Senator_Name = ""
    Sen_Pol = ""
    Sen_State = ""
    Sen_Vote = 0
Master_Dict = {"SENATOR_NAME": Senator_Names, "PARTY":Political_Affiliation,"STATE":State,"Vote":Yea_Nea}

In [ ]:
Master_Dict

# **Answering some questions**

In [ ]:
df = pd.DataFrame.from_dict(Master_Dict)
df

,SENATOR_NAME,PARTY,STATE,Vote
0,Baldwin,D,WI,0
1,Barrasso,R,WY,1
2,Bennet,D,CO,0
3,Blackburn,R,TN,1
4,Blumenthal,D,CT,0
...,...,...,...,...
95,Warren,D,MA,0
96,Whitehouse,D,RI,0
97,Wicker,R,MS,1
98,Wyden,D,OR,0


** What's the total number of Yea votes from all senators whose name starts with "B"? **

In [ ]:
count = 0
for i in df.index:
  if df["SENATOR_NAME"][i][0] == "B":
    count += df["Vote"][i]
count

6

**How many Democrats voted Yea? N**

In [ ]:
count = 0
for i in df.index:
  if df["PARTY"][i] == "D":
    if df["Vote"][i] == 1:
      print(df["SENATOR_NAME"][i])
      count += 1
count

0

**How many states voted all Yea? N**

In [ ]:
count_Nea = 0
count_Yea = 0
lst = []
states_ = df.STATE.unique()
for state in states_:
  for i in df.index:
    if df["STATE"][i] == state:
      lst.append(df["Vote"][i])
  if lst[0] == lst[1] == 0:
    count_Nea += 1
  if lst[0] == lst[1] == 1:
    count_Yea += 1 
  lst = []
print(count_Yea)
print(count_Nea)

22
22


**How many states have both Republican and Democrat senators**

In [ ]:
count = 0
flag1 = 0
flag2 = 0
states_ = df.STATE.unique()
for state in states_:
  for i in df.index:
    if df["STATE"][i] == state:
      if df["PARTY"][i] == "D":
        flag1 = 1
      if df["PARTY"][i] == "R":
        flag2 = 1
  if flag1 == flag2 == 1:
    count += 1
  flag1 = 0
  flag2 = 0       
count

5

**What is the highest absolute difference in percentage between yea and nay senators among the different parties? N**

In [ ]:
D_Y = 0 #Number of democrats who voted Yea
D_N = 0 #Number of democrats who voted Nea
df_D = df[df.PARTY == "D"]
df_D

,SENATOR_NAME,PARTY,STATE,Vote
0,Baldwin,D,WI,0
2,Bennet,D,CO,0
4,Blumenthal,D,CT,0
6,Booker,D,NJ,0
9,Brown,D,OH,0
11,Cantwell,D,WA,0
13,Cardin,D,MD,0
14,Carper,D,DE,0
15,Casey,D,PA,0
18,Coons,D,DE,0


In [ ]:
df_R = df[df.PARTY == "R"]
df_R

,SENATOR_NAME,PARTY,STATE,Vote
1,Barrasso,R,WY,1
3,Blackburn,R,TN,1
5,Blunt,R,MO,1
7,Boozman,R,AR,1
8,Braun,R,IN,1
10,Burr,R,NC,1
12,Capito,R,WV,1
16,Cassidy,R,LA,1
17,Collins,R,ME,1
19,Cornyn,R,TX,1


In [ ]:
df_I = df[df.PARTY == "I"]
df_I

,SENATOR_NAME,PARTY,STATE,Vote
47,King,I,ME,0
75,Sanders,I,VT,0
